In [1]:
import torch
import torch.nn as nn

from torchvision.models.vgg import vgg16

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = vgg16(pretrained=True)

model

c:\Users\user\.conda\envs\han\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.conda\envs\han\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\user/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
import tqdm
import os

from torchvision import datasets, models, transforms
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam

In [7]:
#정규화 Normalization
#데이터의 분포를 정규분포 형태로 바꾸는것
#이미지가 특정 색에 편향되어있으면 학습에 안좋은 영향을 끼칠수 있음

training_data = CIFAR10(
    root='./custom_dataset/train/0/',
    train=True,
    download=True,
    transform=ToTensor()
)


imgs = [item[0] for item in training_data]

In [ ]:
imgs = torch.stack(imgs,dim=0).numpy()

In [ ]:
imgs.shape

In [ ]:
mean_r = imgs[:,0,:,:].mean()
mean_g = imgs[:,1,:,:].mean()
mean_b = imgs[:,2,:,:].mean()

print(mean_r,mean_g,mean_b)

In [ ]:
std_r = imgs[:,0,:,:].std()
std_g = imgs[:,1,:,:].std()
std_b = imgs[:,2,:,:].std()

print(std_r,std_g,std_g)

In [ ]:
fransforms = Compose([
    Resize(224),
    RandomCrop((224,224),padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

In [9]:
# train_data = CIFAR10(
#     root='./',
#     train=True,
#     download=True,
#     transform=fransforms
# )

# test_data = CIFAR10(
#     root='./',
#     train=False,
#     download=True,
#     transform=fransforms
# )

# train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
# test_loader = DataLoader(test_data,batch_size=32,shuffle=True)




transforms_train = transforms.Compose([
    Resize(224),
    RandomCrop((224,224),padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    RandomCrop((224,224),padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)




lr = 0.004
optim = Adam(model.parameters(),lr=lr)

for epoch in range(1):
    iterator = tqdm.tqdm(train_loader)

    for data,label in iterator:
        optim.zero_grad()

        preds = model(data.to(device))
        loss = nn.CrossEntropyLoss()(preds,label.to(device))
        loss.backward()
        optim.step()

        iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(),'cifar10_pre.pth')

epoch:1 loss:0.7634900212287903: 100%|██████████| 105/105 [07:13<00:00,  4.12s/it]


In [14]:

dict_model = torch.load('cifar10_pre.pth',map_location=device)
model.load_state_dict(dict_model)

num_corr = 0


with torch.no_grad():
    for data, label in test_loader:
        output = model(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr

    print(num_corr/len(test_loader))

2.8776315789473683


In [12]:
torch.no_grad()

In [15]:
(num_corr / len(test_datasets)) * 100

71.98815009874919